<a href="https://colab.research.google.com/github/pcamarillor/O2024_ESI3914O/blob/pablo_camarillo_add_spark_setup/examples/notebooks/lab01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!sudo apt update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
# Check this site for the latest download link
# https://www.apache.org/dyn/closer.lua/spark
!wget -q https://dlcdn.apache.org/spark/spark-3.5.2/spark-3.5.2-bin-hadoop3.tgz
!tar xf spark-3.5.2-bin-hadoop3.tgz
!pip install -q findspark
!pip install pyspark
!pip install py4j

Password:
sudo: a password is required
zsh:1: command not found: apt-get
zsh:1: command not found: wget
tar: Error opening archive: Failed to open 'spark-3.5.2-bin-hadoop3.tgz'

[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: /Applications/Xcode.app/Contents/Developer/usr/bin/python3 -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: /Applications/Xcode.app/Contents/Developer/usr/bin/python3 -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: /Applications/Xcode.app/Contents/Developer/usr/bin/python3 -m pip install --upgrade pip


In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
  .master("local[*]") \
  .appName("ITESO-2024-SparkIntroduction") \
  .config("spark.driver.bindAddress","localhost") \
  .config("spark.ui.port","4040") \
  .getOrCreate()

sc = spark.sparkContext

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/08/30 20:40:25 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
# Create an RDD with a list of sentences
sentences = sentences = [
    "Lorem ipsum dolor sit amet, consectetur adipiscing elit.",
    "Sed do eiusmod tempor incididunt ut labore et dolore magna aliqua.",
    "Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat.",
    "Duis aute irure dolor in reprehenderit in voluptate velit esse cillum dolore eu fugiat nulla pariatur.",
    "Excepteur sint occaecat cupidatat non proident, sunt in culpa qui officia deserunt mollit anim id est laborum.",
    "Curabitur pretium tincidunt lacus. Nulla gravida orci a odio.",
    "Nullam varius, turpis et commodo pharetra, est eros bibendum elit, nec malesuada elit elit vel lectus.",
    "Sed ut perspiciatis unde omnis iste natus error sit voluptatem accusantium doloremque laudantium.",
    "Nemo enim ipsam voluptatem quia voluptas sit aspernatur aut odit aut fugit.",
    "Sed quia consequuntur magni dolores eos qui ratione voluptatem sequi nesciunt.",
    "Neque porro quisquam est qui dolorem ipsum quia dolor sit amet, consectetur, adipisci velit.",
    "Lorem ipsum dolor sit amet, consectetur adipiscing elit. Sed do eiusmod tempor incididunt ut labore et dolore magna aliqua.",
    "Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat."
]
sentences_rdd = sc.parallelize(sentences)

# Tokenize the sentences into words
words_rdd = sentences_rdd.flatMap(lambda line: line.split())

print(words_rdd.collect())

['Lorem', 'ipsum', 'dolor', 'sit', 'amet,', 'consectetur', 'adipiscing', 'elit.', 'Sed', 'do', 'eiusmod', 'tempor', 'incididunt', 'ut', 'labore', 'et', 'dolore', 'magna', 'aliqua.', 'Ut', 'enim', 'ad', 'minim', 'veniam,', 'quis', 'nostrud', 'exercitation', 'ullamco', 'laboris', 'nisi', 'ut', 'aliquip', 'ex', 'ea', 'commodo', 'consequat.', 'Duis', 'aute', 'irure', 'dolor', 'in', 'reprehenderit', 'in', 'voluptate', 'velit', 'esse', 'cillum', 'dolore', 'eu', 'fugiat', 'nulla', 'pariatur.', 'Excepteur', 'sint', 'occaecat', 'cupidatat', 'non', 'proident,', 'sunt', 'in', 'culpa', 'qui', 'officia', 'deserunt', 'mollit', 'anim', 'id', 'est', 'laborum.', 'Curabitur', 'pretium', 'tincidunt', 'lacus.', 'Nulla', 'gravida', 'orci', 'a', 'odio.', 'Nullam', 'varius,', 'turpis', 'et', 'commodo', 'pharetra,', 'est', 'eros', 'bibendum', 'elit,', 'nec', 'malesuada', 'elit', 'elit', 'vel', 'lectus.', 'Sed', 'ut', 'perspiciatis', 'unde', 'omnis', 'iste', 'natus', 'error', 'sit', 'voluptatem', 'accusantium'

In [4]:
# Compute the frequency of each word
word_counts_rdd = words_rdd.map(lambda word: (word, 1)).reduceByKey(lambda a, b: a + b)
print(word_counts_rdd.collect())

[('ipsum', 3), ('consectetur', 2), ('laboris', 2), ('nisi', 2), ('aliquip', 2), ('cillum', 1), ('nulla', 1), ('pariatur.', 1), ('sunt', 1), ('anim', 1), ('lacus.', 1), ('aspernatur', 1), ('fugit.', 1), ('porro', 1), ('amet,', 3), ('tempor', 2), ('incididunt', 2), ('ut', 5), ('minim', 2), ('velit', 1), ('qui', 3), ('mollit', 1), ('id', 1), ('a', 1), ('odio.', 1), ('Nullam', 1), ('lectus.', 1), ('voluptatem', 3), ('accusantium', 1), ('ipsam', 1), ('voluptas', 1), ('consequuntur', 1), ('adipisci', 1), ('reprehenderit', 1), ('voluptate', 1), ('sint', 1), ('est', 3), ('pharetra,', 1), ('iste', 1), ('quia', 3), ('ratione', 1), ('Ut', 2), ('eu', 1), ('tincidunt', 1), ('Nulla', 1), ('gravida', 1), ('turpis', 1), ('elit,', 1), ('nec', 1), ('elit', 2), ('natus', 1), ('doloremque', 1), ('laudantium.', 1), ('aut', 2), ('sequi', 1), ('Lorem', 2), ('elit.', 2), ('do', 2), ('exercitation', 2), ('ullamco', 2), ('ex', 2), ('aute', 1), ('in', 3), ('fugiat', 1), ('officia', 1), ('laborum.', 1), ('varius,

In [5]:
# Find the most common word
most_common_word = word_counts_rdd.takeOrdered(1, key=lambda x: -x[1])
print("Most common word:", most_common_word)

Most common word: [('ut', 5)]


In [6]:
# Compute the average word length
total_length_rdd = words_rdd.map(lambda word: len(word)).reduce(lambda a, b: a + b)
total_words = words_rdd.count()
average_word_length = total_length_rdd / total_words if total_words > 0 else 0
print("Average word length:", average_word_length)

Average word length: 5.605555555555555


In [7]:
# Longest
longest = words_rdd.map(lambda word: (word, len(word))).sortBy(lambda x: x[1], ascending=False).first()
print("Longest word:", longest)

# Shortest
shortest = words_rdd.map(lambda word: (word, len(word))).sortBy(lambda x: x[1]).first()
print("Shortest word:", shortest)

Longest word: ('reprehenderit', 13)
Shortest word: ('a', 1)


In [8]:
# Stop the SparkContext
sc.stop()